# Parameters

In [1]:
from pyspark.conf import SparkConf

In [2]:
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "4g"))


In [3]:
# cols to query in main reviews dataset
REV_COLS = [
    'normalized_url',
    'review_date',
    'review_content',

]

# cols to query in later cell
GROUP_COLS = [
    'normalized_url',
    'review_source_id',
    'review_content',
#   'source_name', # actually some of them are duplicates across sources
    'top_asp',
    ]

# Assumes Spark 2.4

Some array function only in spark >2.3. I use docker image with statsmodels and plotly and vaderSentiment based on spark 2.4.5
Please deloy cluster with `--docker-tag cici-nlp-sentiment` or other spark 2.4 docker image.

In [4]:
sc

<SparkContext master=spark://terry.cu:7077 appName=PySparkShell>

# Imports

In [5]:
%%capture
!pip install watermark
!pip3 install --upgrade --no-cache-dir --extra-index-url http://pypi.cu/root/circleup/+simple/ --trusted-host pypi.cu cu-helio-insights==0.0.12
!pip install -Iv numpy==1.17.2
!pip install ipynb

In [6]:
%load_ext watermark
%watermark
%watermark -v -m --iversions -g

2022-07-07T20:45:19+00:00

CPython 3.6.8
IPython 6.1.0

compiler   : GCC 6.3.0 20170516
system     : Linux
release    : 4.9.43-17.39.amzn1.x86_64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit
py4j     0.10.7
platform 1.0.8
CPython 3.6.8
IPython 6.1.0

compiler   : GCC 6.3.0 20170516
system     : Linux
release    : 4.9.43-17.39.amzn1.x86_64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit
Git hash   :


In [7]:
%%capture
!pip install vaderSentiment

In [54]:
import numpy as np
import pandas as pd
from pyspark.sql.window import Window
#from pyspark.sql.functions import arrays_zip
from pyspark.ml.feature import CountVectorizer, IDF, HashingTF
from pyspark.ml.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import string
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn import linear_model
import numpy as np
import statsmodels.api as sm
from itertools import chain
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

import insights
from insights.investor_tools.widgets.style import (
    Font,
    CU_PLOTLY_COLOR_SEQUENCE,
    CU_PLOTLY_COLORSCALE,
    Color
)
from spark_tools import T, F, c, read_google_sheet
import apollo
from apollo import OverrideConfiguration
from apollo import dataset
import apollo_artifacts
from apollo_artifacts import datasets
from transform.attributes.utils import tf_idf_attributes
import builtins

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import decomposition

# Import stopwords and functions .ipynb from HAI General S3

Much of the logic for pulling and cleaning reviews is shared between TF-IDF and KPC scripts

In [9]:
os.getcwd()

'/circleup-notebooks/s3-circleup-notebooks/tzeng'

In [10]:
# cd to where the helpers and config files are located
%cd /circleup-notebooks/s3-circleup-notebooks/HAI/General/Code/TF_IDF

from ipynb.fs.full.functions_tfidf import *

# cd - command changes back to previously defined directory (should be same as og_filepath)
%cd 
%pwd

/circleup-notebooks/s3-circleup-notebooks/HAI/General/Code/TF_IDF
/root


'/root'

In [11]:
# cd to where the helpers and config files are located
%cd /circleup-notebooks/s3-circleup-notebooks/HAI/General/helpers

from ipynb.fs.full.general_helpers import *

# cd - command changes back to previously defined directory (should be same as og_filepath)
%cd 
%pwd

/circleup-notebooks/s3-circleup-notebooks/HAI/General/helpers
/root


'/root'

# Settings

In [12]:
np.random.seed(42) # set seed for models for reproducibility

stopwords_li = stopwords.words('english')
punkts_li = list(string.punctuation)

pd.options.display.max_rows = 300
OverrideConfiguration(default_to_production=True).apply()

spark.conf.set('spark.sql.execution.arrow.enabled', 'false')

# Load Data

In [13]:
from datetime import *
from datetime import datetime, timedelta


test_timeline_data = (
apollo
.dataset('online_reviews__deduped_review__1_0')
.read()
.select('normalized_url', 'execution_date')
.where(c.execution_date > date.today() - timedelta(days=7))
)

max_date = test_timeline_data.where(c.normalized_url.isNotNull()).agg({'execution_date':'max'}).collect()[0][0]

reviews = (
dataset('online_reviews__deduped_review__1_0')
.read_segment_df({'execution_date': max_date})
.select(REV_COLS)
.persist()
)

In [14]:
categories = (
dataset('helio_summary__2_0').read()
.select(["normalized_url","parent_category","primary_category"])
.persist()
)

In [15]:
#bypass circleup limit

reviews = reviews.join(categories, ["normalized_url"], "left").filter(F.col("primary_category").isin(["Skin Care"])).distinct()
raw_1 = reviews.filter(F.col("review_date").between(datetime.strptime('2021-07-01 12:00:00', '%Y-%m-%d %H:%M:%S'),datetime.strptime('2022-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')))
raw_2 = reviews.filter(F.col("review_date").between(datetime.strptime('2020-07-01 12:00:00', '%Y-%m-%d %H:%M:%S'),datetime.strptime('2021-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')))
raw_3 = reviews.filter(F.col("review_date").between(datetime.strptime('2019-07-01 12:00:00', '%Y-%m-%d %H:%M:%S'),datetime.strptime('2020-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')))
raw_4 = reviews.filter(F.col("review_date").between(datetime.strptime('2018-07-01 12:00:00', '%Y-%m-%d %H:%M:%S'),datetime.strptime('2019-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')))
raw_5 = reviews.filter(F.col("review_date").between(datetime.strptime('2017-07-01 12:00:00', '%Y-%m-%d %H:%M:%S'),datetime.strptime('2018-07-01 00:00:00', '%Y-%m-%d %H:%M:%S')))

In [16]:
raw_1_df = raw_1.drop("review_date","primary_category","parent_category").toPandas()
#raw_2_df = raw_2.drop("review_date","primary_category","parent_category").toPandas()
#raw_3_df = raw_3.drop("review_date","primary_category","parent_category").toPandas()
#raw_4_df = raw_4.drop("review_date","primary_category","parent_category").toPandas()
#raw_5_df = raw_5.drop("review_date","primary_category","parent_category").toPandas()

In [17]:
raw_1_df

,normalized_url,review_content
0,albolene.com,I use this product for initial facial cleansi...
1,albolene.com,I have tried dozens of make-up removers and A...
2,albolene.com,Imagine blue magic or Vaseline rubbed all ove...
3,albolene.com,I have been using this product for years. I u...
4,albolene.com,I've been using the classic Nivea tin (German...
5,albolene.com,"To give some context, I have very sensitive, ..."
6,albolene.com,I have used Albolene for more than 25 years f...
7,albolene.com,I am sixty and this moisturizer cleanser can ...
8,albolene.com,My mother used this product for years. Her sk...
9,albolene.com,It will get dirt that soap and water can't


# Clean Data

In [18]:
raw_1_df["review_content_cleaned"] = raw_1_df["review_content"].map(lambda x: re.sub('[,\.!?]', '', str(x))) #remove punctuation
raw_1_df["review_content_cleaned"] = raw_1_df["review_content_cleaned"].map(lambda x:x.lower()) #lowercase

In [19]:
#drop words irrelevant to analysis

# LDA

In [34]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def tokenize (text):
    tokens = [word for word in nltk.word_tokenize(text) if len(word)>3]
    return tokens

In [44]:
x_train, xhold = train_test_split(raw_1_df, test_size = 0.5)

In [49]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features = 10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(x_train["review_content_cleaned"])

In [52]:
vectorizer_tf.get_feature_names()

['*gifted',
 '-the',
 '1-2x',
 '10-12',
 '10-15',
 '10/10',
 '100+',
 '1000',
 '10000',
 '100ml',
 '10mg',
 '10th',
 '12-hexanediol',
 '1200',
 '15-20',
 '1500',
 '16oz',
 '2-3x',
 '2-in-1',
 '2-pack',
 '20-30',
 '2000',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '24/7',
 '24hrs',
 '2500',
 '3-in-1',
 '3-pack',
 '3000',
 '30ml',
 '30’s',
 '4-in-1',
 '5-10',
 '5-star',
 '50/50',
 '5000',
 '500mg',
 '50ml',
 '8-10',
 '8/10',
 '9/10',
 '_blank',
 'a+++',
 'abdomen',
 'abdominal',
 'aber',
 'abilities',
 'ability',
 'able',
 'abrasions',
 'abrasive',
 'abreva',
 'absolute',
 'absolutely',
 'absorb',
 'absorbable',
 'absorbe',
 'absorbed',
 'absorbency',
 'absorbent',
 'absorbing',
 'absorbs',
 'absorption',
 'abuse',
 'acabado',
 'accent',
 'accents',
 'accentuate',
 'accept',
 'acceptable',
 'accepted',
 'accesible',
 'access',
 'accessible',
 'accessories',
 'accessory',
 'accident',
 'accidental',
 'accidentally',
 'accidently',
 'accidents',
 'accommodate'

In [57]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method="online", learning_offset = 50)

w1 = lda.fit_transform(tf_vectors)
h1 = lda.components_

In [58]:
w1

array([[0.67465144, 0.02083333, 0.0209345 , 0.02083333, 0.24185712,
        0.02089027],
       [0.32307024, 0.54820458, 0.11348717, 0.00506089, 0.00507358,
        0.00510354],
       [0.00795695, 0.66921567, 0.00799204, 0.07255874, 0.00794632,
        0.23433028],
       ...,
       [0.04217247, 0.04212844, 0.33002569, 0.50168484, 0.0416679 ,
        0.04232067],
       [0.01284358, 0.01282771, 0.34010809, 0.25097698, 0.37033033,
        0.0129133 ],
       [0.01520219, 0.35470267, 0.47650463, 0.01523366, 0.01528253,
        0.12307432]])

In [63]:
num_words=15
vocab = np.array(vectorizer_tf.get_feature_names())
top_words = lambda t:[vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in h1])
topics = [' '.join(t) for t in topic_words]

In [64]:
topics

['review promotion collected product serum smell received free strong scent ingredients really like sample spray',
 'skin love product like face great really hair feel does good it’s soft nice feels',
 'good great product like price love works just really water brand easy taste better it’s',
 'love lips loves color mascara lashes affordable large nails ones perfectly cats cheap easy easier',
 'size just easy perfect like great small bought nice came pain little ordered disappointed gift',
 'product using used years just i’ve eyes time difference tried really results worth days week']

In [ ]:
#Topics interpreted
#1. Scent
#2. Feel on skin
#3. Price
#4. Color
#5. Packaging
#6. Efficacy

In [68]:
colnames = ["Topic" + str(i) for i in range (lda.n_components)]
docnames = ["Review" + str(i) for i in range(len(x_train.review_content_cleaned))]
df_doc_topic = pd.DataFrame(np.round(w1,2), columns= colnames, index = docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant topic'] = significant_topic

In [70]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant topic
Review0,0.67,0.02,0.02,0.02,0.24,0.02,0
Review1,0.32,0.55,0.11,0.01,0.01,0.01,1
Review2,0.01,0.67,0.01,0.07,0.01,0.23,1
Review3,0.01,0.01,0.53,0.01,0.01,0.42,2
Review4,0.01,0.43,0.01,0.51,0.01,0.01,3
Review5,0.02,0.02,0.91,0.02,0.02,0.02,2
Review6,0.02,0.47,0.02,0.02,0.02,0.46,1
Review7,0.08,0.08,0.58,0.08,0.08,0.08,2
Review8,0.04,0.38,0.36,0.01,0.01,0.19,1
Review9,0.04,0.04,0.04,0.04,0.04,0.79,5


In [71]:
x_train.head()

,normalized_url,review_content,review_content_cleaned
725823,dove.com,I was all in on the concept but the actual deo...,i was all in on the concept but the actual deo...
625826,hawaiiantropic.com,(This review was collected as part of a promot...,(this review was collected as part of a promot...
2774830,olay.com,This is a great addition to washing your face...,this is a great addition to washing your face...
1326271,lorealparisusa.com,L’Oreal Paris detox in Brighton facemask is a ...,l’oreal paris detox in brighton facemask is a ...
3889481,littlegiant.com,Love the functionality of this. It’s so much ...,love the functionality of this it’s so much e...
